# Making Your Model Learn Addition!
## Part 1: Introduction

Given the string "54+7", the model should return a prediction: "61".

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

print('Tested with tensorflow version 2.0.1')
print('Using tensorflow version:', tf.__version__)

Tested with tensorflow version 2.0.1
Using tensorflow version: 1.12.0


## Part 2: Generate Data
___

In [2]:
all_chars = '0123456789+'

In [3]:
num_features = len(all_chars)

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

print('Number of features:', num_features)

Number of features: 11


In [4]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    example = str(first_num) + '+' + str(second_num)
    label = str(first_num+second_num)
    return example, label

generate_data()

('38+74', '112')

## Part 3: Create the Model
___
Consider these two reviews:

Review 1: This movie is not terrible at all.

Review 2: This movie is pretty decent.

In [5]:
hidden_units = 128
max_time_steps = 5

model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               17920     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


## Part 4: Vectorize and De-Vectorize Data
___


In [6]:
def vectorize_example(example, label):
    
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 99+60 159
Vectorized Example and Label Shapes: (5, 11) (5, 11)


In [7]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

devectorize_example(x)

'99+60'

In [8]:
devectorize_example(y)

'00159'

## Part 5: Create Dataset
___

In [9]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, max_time_steps, num_features))
    y_train = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset(20000)
print(x_train.shape, y_train.shape)

(20000, 5, 11) (20000, 5, 11)


In [10]:
devectorize_example(x_train[0])

'08+89'

In [11]:
devectorize_example(y_train[0])

'00097'

## Part 6: Training the Model
___
Please note that the hyper-parameters are a little different. This model should easily get to 100% accuracy.

In [12]:
simple_logger = LambdaCallback(
    on_epoch_end=lambda e, l: print('{:.2f}'.format(l['val_acc']), end=' _ ')
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=False,
         batch_size=1024, callbacks=[simple_logger, early_stopping])

0.61 _ 0.62 _ 0.63 _ 0.64 _ 0.67 _ 0.68 _ 0.71 _ 0.72 _ 0.75 _ 0.76 _ 0.77 _ 0.78 _ 0.78 _ 0.78 _ 0.78 _ 0.80 _ 0.80 _ 0.80 _ 0.78 _ 0.81 _ 0.81 _ 0.82 _ 0.80 _ 0.82 _ 0.83 _ 0.84 _ 0.85 _ 0.86 _ 0.87 _ 0.88 _ 0.90 _ 0.91 _ 0.93 _ 0.94 _ 0.95 _ 0.95 _ 0.96 _ 0.97 _ 0.97 _ 0.98 _ 0.98 _ 0.98 _ 0.98 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 

Create a test set and look at some predictions:

In [13]:
x_test, y_test = create_dataset(num_examples=20)
preds = model.predict(x_test)
full_seq_acc = 0

for i, pred in enumerate(preds):
    pred_str = devectorize_example(pred)
    y_test_str = devectorize_example(y_test[i])
    x_test_str = devectorize_example(x_test[i])
    col = 'green' if pred_str == y_test_str else 'red'
    full_seq_acc += 1/len(preds) * int(pred_str == y_test_str)
    outstring = 'Input: {}, Out: {}, Pred: {}'.format(x_test_str, y_test_str, pred_str)
    print(colored(outstring, col))
print('\nFull sequence accuracy: {:.3f} %'.format(100 * full_seq_acc))

Input: 80+82, Out: 00162, Pred: 00162
Input: 42+58, Out: 00100, Pred: 00100
Input: 72+42, Out: 00114, Pred: 00114
Input: 51+73, Out: 00124, Pred: 00124
Input: 26+55, Out: 00081, Pred: 00081
Input: 84+56, Out: 00140, Pred: 00140
Input: 58+82, Out: 00140, Pred: 00140
Input: 27+84, Out: 00111, Pred: 00111
Input: 69+40, Out: 00109, Pred: 00109
Input: 61+85, Out: 00146, Pred: 00146
Input: 15+98, Out: 00113, Pred: 00113
Input: 87+35, Out: 00122, Pred: 00122
Input: 37+28, Out: 00065, Pred: 00065
Input: 001+1, Out: 00002, Pred: 00002
Input: 050+7, Out: 00057, Pred: 00057
Input: 51+14, Out: 00065, Pred: 00065
Input: 75+44, Out: 00119, Pred: 00119
Input: 16+90, Out: 00106, Pred: 00106
Input: 36+87, Out: 00123, Pred: 00123
Input: 72+86, Out: 00158, Pred: 00158

Full sequence accuracy: 100.000 %
